In [0]:
%run ./_common

In [0]:
config_values = [
    ("catalog", "provider_catalog"),
    ("schema", "retail"),
    ("internal_share_name", "internal_retail"),
    ("internal_recipient_name", "west_division"),
    ("external_share_name", "partner_retail"),
    ("external_recipient_name", "partner_co"),
]
DA = create_da_objects(config_values)

In [ ]:
# Test in Python
print("Python DA dictionary:")
print(f"Catalog: {DA['catalog']}")
print(f"Schema: {DA['schema']}")
print(f"Internal Share: {DA['internal_share_name']}")
print(f"Internal Recipient: {DA['internal_recipient_name']}")
print(f"External Share: {DA['external_share_name']}")
print(f"External Recipient: {DA['external_recipient_name']}")
print("\nFull DA dict:")
print(DA)

In [ ]:
%sql
-- Test in SQL
SELECT DA;

-- Test accessing individual values
SELECT DA['catalog'] as catalog,
       DA['schema'] as schema,
       DA['internal_share_name'] as internal_share,
       DA['internal_recipient_name'] as internal_recipient,
       DA['external_share_name'] as external_share,
       DA['external_recipient_name'] as external_recipient;

-- Test using in SQL statements
SELECT 'CREATE CATALOG IF NOT EXISTS ' || DA['catalog'] as create_catalog_sql;
SELECT 'USE CATALOG ' || DA['catalog'] as use_catalog_sql;

In [ ]:
# set python variables
DA = {}
DA.catalog = ""
DA.schema = 
DA.internal_share_name = "internal_retail"
DA.internal_recipient_name = ""
DA.external_share_name = ""
DA.external_recipient_name = ""


In [ ]:
acme corp
east division
west division

partner corp

# set python variables
DA = {}
DA.catalog = "acme_corp"
DA.schema = "retail"
DA.internal_share_name = "internal_retail"
DA.internal_recipient_name = "west_division"
DA.external_share_name = "external_retail"
DA.external_recipient_name = "partner_analytics"

config_values = [
    ("catalog", DA["catalog"]),
    ("schema", DA["schema"]),
    ("internal_share_name", DA["internal_share_name"]),
    ("internal_recipient_name", DA["internal_recipient_name"]),
    ("external_share_name", DA["external_share_name"]),
    ("external_recipient_name", DA["external_recipient_name"]),    
]

In [ ]:
# set SQL variables
declare_variable = 'DECLARE OR REPLACE DA MAP<STRING, STRING>'
spark.sql(declare_variable)

## Set the variable with literal map values
set_variable = f'''
SET VAR DA = map(
    'catalog', '{DA.catalog}',
    'schema', '{DA.schema}',
    'internal_share_name', '{DA.share_name}',
    'internal_recipient_name', '{DA.recipient_name}',
    'external_share_name', '{DA.share_name}',
    'external_recipient_name', '{DA.recipient_name}'    
)
'''
spark.sql(set_variable)

In [ ]:
# create catalog if not exists
create_catalog = f'''
CREATE CATALOG IF NOT EXISTS {DA.catalog}
'''
spark.sql(create_catalog)

In [ ]:
%sql
-- Create customers table
CREATE OR REPLACE TABLE ${c.catalog_name}.${c.schema_name}.customers (
  customer_id INT,
  customer_name STRING,
  email STRING,
  country STRING,
  signup_date DATE,
  customer_segment STRING
)
COMMENT 'Customer master data';

-- Insert sample data
INSERT INTO ${c.catalog_name}.${c.schema_name}.customers VALUES
  (1, 'Acme Corp', 'contact@acme.com', 'USA', '2023-01-15', 'Enterprise'),
  (2, 'TechStart Inc', 'info@techstart.com', 'Canada', '2023-02-20', 'SMB'),
  (3, 'Global Solutions', 'hello@globalsol.com', 'UK', '2023-03-10', 'Enterprise'),
  (4, 'Innovate Ltd', 'contact@innovate.com', 'Australia', '2023-04-05', 'Mid-Market'),
  (5, 'Data Dynamics', 'team@datadyn.com', 'Germany', '2023-05-12', 'Enterprise'),
  (6, 'Cloud First', 'info@cloudfirst.com', 'USA', '2023-06-18', 'SMB'),
  (7, 'NextGen Analytics', 'contact@nextgen.com', 'Singapore', '2023-07-22', 'Mid-Market'),
  (8, 'Digital Transform', 'hello@digitrans.com', 'France', '2023-08-30', 'Enterprise');

In [ ]:
%sql
-- Create sales transactions table
CREATE OR REPLACE TABLE ${c.catalog_name}.${c.schema_name}.sales_transactions (
  transaction_id INT,
  customer_id INT,
  product_name STRING,
  quantity INT,
  unit_price DECIMAL(10,2),
  total_amount DECIMAL(10,2),
  transaction_date DATE,
  region STRING
)
COMMENT 'Sales transaction data';

-- Insert sample sales data
INSERT INTO ${c.catalog_name}.${c.schema_name}.sales_transactions VALUES
  (101, 1, 'Data Platform License', 5, 1000.00, 5000.00, '2024-01-15', 'North America'),
  (102, 2, 'Analytics Suite', 2, 500.00, 1000.00, '2024-01-20', 'North America'),
  (103, 3, 'Data Platform License', 10, 1000.00, 10000.00, '2024-02-05', 'EMEA'),
  (104, 1, 'Storage Add-on', 3, 200.00, 600.00, '2024-02-10', 'North America'),
  (105, 4, 'Analytics Suite', 5, 500.00, 2500.00, '2024-02-15', 'APAC'),
  (106, 5, 'Data Platform License', 15, 1000.00, 15000.00, '2024-03-01', 'EMEA'),
  (107, 3, 'Storage Add-on', 8, 200.00, 1600.00, '2024-03-10', 'EMEA'),
  (108, 6, 'Analytics Suite', 1, 500.00, 500.00, '2024-03-15', 'North America'),
  (109, 7, 'Data Platform License', 7, 1000.00, 7000.00, '2024-04-01', 'APAC'),
  (110, 8, 'Storage Add-on', 10, 200.00, 2000.00, '2024-04-05', 'EMEA');

In [ ]:
display_config_values(config_values)

In [0]:
%python
# Get current user information
current_user = spark.sql("SELECT current_user() as user").collect()[0]["user"]
username = current_user.split("@")[0].replace(".", "_").replace("-", "_")

# Provider configuration
provider_catalog = f"{username}_provider_catalog"
schema_name = "sales_data"
share_name = f"{username}_customer_share"
cdf_share_name = f"{username}_cdf_share"
d2d_recipient = f"{username}_d2d_recipient"
d2o_recipient = f"{username}_d2o_recipient"

# Set as Spark configuration for SQL access
spark.conf.set("c.catalog_name", provider_catalog)
spark.conf.set("c.schema_name", schema_name)
spark.conf.set("c.share_name", share_name)
spark.conf.set("c.cdf_share_name", cdf_share_name)
spark.conf.set("c.d2d_recipient", d2d_recipient)
spark.conf.set("c.d2o_recipient", d2o_recipient)

print("="*80)
print("PROVIDER WORKSPACE CONFIGURATION")
print("="*80)
print(f"Current User: {current_user}")
print(f"Username: {username}")
print(f"")
print(f"Catalog: {provider_catalog}")
print(f"Schema: {schema_name}")
print(f"Share (D2D): {share_name}")
print(f"Share (CDF): {cdf_share_name}")
print(f"Recipient (D2D): {d2d_recipient}")
print(f"Recipient (D2O): {d2o_recipient}")
print("="*80)
print("✅ Variables configured! Use ${c.variable_name} in SQL cells")
print("="*80)

## Verify Configuration

The variables are now available in SQL using the `${c.variable_name}` syntax

In [0]:
SELECT 
  '${c.catalog_name}' as catalog_name,
  '${c.schema_name}' as schema_name,
  '${c.share_name}' as share_name,
  '${c.d2d_recipient}' as d2d_recipient,
  current_user() as current_user